In [ ]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob
from tensorflow.keras.layers import *
import cv2
import pickle
from scipy.ndimage import median_filter, convolve
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import sklearn

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
ls '/content/drive/My Drive/flowers/'

daisy/  dandelion/  flowers/  rose/  sunflower/  tulip/


In [ ]:
data_dir = "/content/drive/My Drive/flowers"

categories = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

In [ ]:
data = []
def make_data():
    for category in categories:
        path = os.path.join(data_dir, category)
        label = categories.index(category)
        
        for img_name in os.listdir(path):
            image_path = os.path.join(path, img_name)
            image = cv2.imread(image_path)
            
            try:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, (224,224))
                
                image = np.array(image, dtype=np.float32)
                
                data.append([image, label])
            except Exception as e:
                pass
    print(len(data))
    
    pik = open('data.pickle', 'wb')
    pickle.dump(data,pik)
    pik.close()

In [ ]:
make_data()

4346


In [ ]:
def load_data():
    pick = open('data.pickle','rb')
    data = pickle.load(pick)
    pick.close()
    
    np.random.shuffle(data)
    
    feature = []
    labels = []
    
    for img, label in data:
        feature.append(img)
        labels.append(label)
        
    feature = np.array(feature, dtype=np.float32)
    labels = np.array(labels)
    
    feature = feature/255.0
    
    return [feature, labels]

In [ ]:
(feature, labels) = load_data()

In [ ]:
labels

array([0, 4, 4, ..., 4, 1, 0])

In [ ]:

labels = labels.reshape((-1,1))

In [ ]:

labels.shape

(4346, 1)

In [ ]:


x_train, x_test, y_train, y_test = train_test_split (feature, labels, test_size = 0.1)

In [ ]:
x_train.shape

(3911, 224, 224, 3)

In [ ]:
y_test.shape

(435, 1)

In [ ]:
input_shape = (224,224,3)
input = tf.keras.layers.Input(input_shape, dtype = tf.uint8)
x = tf.cast(input, tf.float32)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
#x = tf.keras.layers.experimental.preprocessing.RandomRotation(0.5)(x)
#x = tf.keras.layers.experimental.preprocessing.RandomFlip()(x)

base_model = tf.keras.applications.MobileNetV2(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=input_shape,
    include_top=False)
base_model.trainable = False
x = base_model(input, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
output = tf.keras.layers.Dense(5)(x)
output = tf.keras.layers.Softmax()(output)

model = tf.keras.Model(input, output)
model.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_3 ( (None, 1280)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 6405      
_________________________________________________________________
softmax_3 (Softmax)          (None, 5)                 0         
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
________________________________________

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

In [ ]:
batch_size = 64
epochs = 40
patience = 10
model.fit(x=x_train,y=y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), 
          callbacks = [tf.keras.callbacks.EarlyStopping(patience=patience)])

Epoch 1/40
62/62 [==============================] - 11s 183ms/step - loss: 1.6589 - accuracy: 0.2692 - val_loss: 1.5441 - val_accuracy: 0.3218
Epoch 2/40
62/62 [==============================] - 10s 169ms/step - loss: 1.5834 - accuracy: 0.2953 - val_loss: 1.5322 - val_accuracy: 0.3172
Epoch 3/40
62/62 [==============================] - 10s 168ms/step - loss: 1.5462 - accuracy: 0.3178 - val_loss: 1.4952 - val_accuracy: 0.3333
Epoch 4/40
62/62 [==============================] - 10s 168ms/step - loss: 1.5246 - accuracy: 0.3393 - val_loss: 1.4808 - val_accuracy: 0.3563
Epoch 5/40
62/62 [==============================] - 10s 168ms/step - loss: 1.5070 - accuracy: 0.3429 - val_loss: 1.4880 - val_accuracy: 0.3609
Epoch 6/40
62/62 [==============================] - 10s 169ms/step - loss: 1.4903 - accuracy: 0.3513 - val_loss: 1.4740 - val_accuracy: 0.3586
Epoch 7/40
62/62 [==============================] - 10s 168ms/step - loss: 1.4794 - accuracy: 0.3631 - val_loss: 1.4972 - val_accuracy: 0.3632

In [ ]:
# zapisywanie modelu do pliku na colabie
model.save('outdated_network.h5')

In [ ]:
# zapisywanie modelu na komputer lokalnie
from google.colab import files
files.download('outdated_network.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
y_pred.shape

(435, 5)

In [ ]:
y_pred

array([[0.21027134, 0.235938  , 0.25323692, 0.16677302, 0.13378072],
       [0.3398712 , 0.25793695, 0.17218776, 0.11879643, 0.11120773],
       [0.42159814, 0.12799104, 0.26610547, 0.08451737, 0.09978797],
       ...,
       [0.3398712 , 0.25793695, 0.17218776, 0.11879643, 0.11120773],
       [0.19211742, 0.12903082, 0.12349488, 0.31291875, 0.24243814],
       [0.44433546, 0.13150671, 0.13171215, 0.15582232, 0.13662332]],
      dtype=float32)

In [ ]:
x_test.shape

(435, 224, 224, 3)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_preds.argmax(1)))

In [ ]:
y_test.shape

(435, 1)

In [ ]:
sklearn.metrics.confusion_matrix(y_test, y_pred.argmax(1))

array([[68, 11,  0,  2,  1],
       [77, 22,  0,  1,  0],
       [58, 17,  0,  2,  1],
       [63, 14,  0,  0,  0],
       [75, 20,  1,  2,  0]])